Primero cargaremos todo las librerias que usaremos y tambien el archivo.csv de canciones

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from sklearn.neighbors import NearestNeighbors

songs= pd.read_csv('../spotify_songs.csv')
songs.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


Cargamos el csv de vector caracteristico de las canciones

In [2]:
features = pd.read_csv('../new_features2.csv')
features.head()

,track_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,-0.127395,0.062021,-0.143093,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,0ZfM5XfJTtFPhOxAERRnNY,-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,0.214938,0.311463,0.014895,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0zG6VemmYlMhAK5MNvBeP7,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,-0.322022,-0.157966,-0.272551,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,0ZGLuduCPjgWY1n85ykgMe,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,0.326603,0.131136,-0.333797,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0ZgTez1pojM5aCxsV0hrC9,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,-0.055531,0.009561,-0.095176,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317


Verficamos que tenemos lo necesario par auna busqueda textual con pandas solo extraemos las columnas "track_name", "track_artist", "lyrics","track_id". Esto no es necesario, pero se implemento con KNN.

In [3]:
essential = songs[["track_name", "track_artist", "lyrics","track_id"]]
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Limpieza de datos nulos y duplicados

In [4]:
essential = essential.dropna()
essential = essential.drop_duplicates()
essential.shape
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Obtenemos el tfidvectorizer de nuestra dataframe

In [5]:
vectorizer = TfidfVectorizer()
vectorizer
X = vectorizer.fit_transform(essential["lyrics"])
X

<18194x102102 sparse matrix of type '<class 'numpy.float64'>'
	with 2120968 stored elements in Compressed Sparse Row format>

KNN secuencial de busqueda textual implementada

In [6]:
def KNN_secuencial(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = results.argsort()[-k:][::-1]
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs
query = "I'm a rebel just for kicks, now"
k=5
print(KNN_secuencial(query, k))




                                 track_name       track_artist
15794                         Feel It Still  Portugal. The Man
17317                         Feel It Still  Portugal. The Man
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
12584           Rebel Rebel - 1997 Remaster        David Bowie


Tambien se implemento una variacion con heap

In [7]:
import heapq
def KNN_secuencial_heap(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = heapq.nlargest(k, range(len(results)), results.take)
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs

print(KNN_secuencial_heap(query, k))

                                 track_name       track_artist
15794                         Feel It Still  Portugal. The Man
17317                         Feel It Still  Portugal. The Man
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie


Se implemento una busqueda textual con knn_range_search, no es necesario

In [8]:
radius = 0.2
def KNN_range_Search(query, radius):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X, query_vec).reshape((-1,))
    indices = np.where(results >= radius)[0]
    songs = essential.iloc[indices][["track_name", "track_artist"]]
    return songs

print(KNN_range_Search(query, radius))

                                 track_name       track_artist
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
9937                              Pumped Up          Klingande
11489                               McQueen          TrifeDrew
12584           Rebel Rebel - 1997 Remaster        David Bowie
14385           Rebel Rebel - 1999 Remaster        David Bowie
15399                             Pumped Up          Klingande
15720           Rebel Rebel - 2014 Remaster        David Bowie
15786              Rebel Rebel - Remastered        David Bowie
15794                         Feel It Still  Portugal. The Man
16985                         Teenage Kicks     The Undertones
17317                         Feel It Still  Portugal. The Man
18306                       Pumped Up Kicks  Foster The People


In [9]:
first_value = songs.loc[0]
print(first_value)

track_id                                               0017A6SJgTbfQVU2EtsPNo
track_name                                                           Pangarap
track_artist                                                  Barbie's Cradle
lyrics                      Minsan pa Nang ako'y napalingon Hindi ko alam ...
track_popularity                                                           41
track_album_id                                         1srJQ0njEQgd8w4XSqI4JQ
track_album_name                                                         Trip
track_album_release_date                                           2001-01-01
playlist_name                                              Pinoy Classic Rock
playlist_id                                            37i9dQZF1DWYDQ8wBxd7xt
playlist_genre                                                           rock
playlist_subgenre                                                classic rock
danceability                                                    

Esta parte es lo que se pidio implementar, extraemos las mismas columnas importantes "track_name", "track_artist", "lyrics","track_id" para verificar si resulto con existe vemos las primeras filas con .head() de nuestra dataframe.

In [10]:
essential_2 = songs[['track_id', 'track_name', 'track_artist', 'lyrics']]
essential_2.head()

,track_id,track_name,track_artist,lyrics
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu..."
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U..."
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...


Limpieza de datos:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**

Verificamos la cantidad de elementos se redujo o se mantuvo

In [11]:
essential_2 = essential_2.dropna()
essential_2 = essential_2.drop_duplicates()
essential_2.shape


(18194, 4)

Hacemos el merge entre nuestra dataframe extraida essential2 y nuestro csv con el vector caracteristico cargado anteriormente.


In [12]:
combined_data = pd.merge(essential_2, features, on='track_id')
combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15107,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.602929,-0.277219,-0.493499,0.094192,0.304649,-0.014173,...,-0.193526,-0.226324,0.042981,0.040540,0.163899,0.242983,0.437682,0.234670,-0.081395,-1.041992
15108,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.451623,-0.234071,-0.159290,-0.161218,-0.095926,-0.051664,...,-0.119346,-0.139924,-0.272772,-0.386242,0.218397,0.275528,0.536367,0.256531,0.399271,-0.691504
15109,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",0.656685,0.437153,0.526300,0.198158,0.008960,-0.278656,...,-0.006828,-0.205763,0.011333,0.094089,0.197834,0.163011,0.098527,-0.196720,-0.258613,0.252062
15110,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",0.884435,0.957433,0.266550,-0.403613,-0.411181,0.208359,...,0.028934,0.065975,0.331138,0.341994,0.015794,-0.213025,-0.156314,-0.068632,-0.226346,-0.478887


Limipieza de datos luego del merge:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**



In [13]:
combined_data = combined_data.dropna() #Elimino la tabla con valores nulos
combined_data = combined_data.drop_duplicates() #Elimino los duplicados

combined_data.shape

(15112, 364)

Ahora realizaremos la normalizacion del vector caracteristico para eso de la dataframe mezclada solo eligiremos columnas pertenecientes al vector caracteristico:
feature1 hasta feature50

In [14]:

# Seleccionar solo las columnas numéricas
features_to_normalize = combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
features_to_normalize.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,-0.127395,0.062021,-0.143093,0.101124,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,0.214938,0.311463,0.014895,-0.119139,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,-0.322022,-0.157966,-0.272551,-0.103481,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,0.326603,0.131136,-0.333797,0.013289,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,-0.055531,0.009561,-0.095176,0.091638,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317


Alineamos las filas con el conjunto de datos

In [15]:
features_to_normalize = features_to_normalize.loc[combined_data.index]
features_to_normalize



,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,-0.127395,0.062021,-0.143093,0.101124,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,0.214938,0.311463,0.014895,-0.119139,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,-0.322022,-0.157966,-0.272551,-0.103481,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,0.326603,0.131136,-0.333797,0.013289,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,-0.055531,0.009561,-0.095176,0.091638,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15107,-0.602929,-0.277219,-0.493499,0.094192,0.304649,-0.014173,0.122051,0.127419,0.100785,0.084170,...,-0.193526,-0.226324,0.042981,0.040540,0.163899,0.242983,0.437682,0.234670,-0.081395,-1.041992
15108,-0.451623,-0.234071,-0.159290,-0.161218,-0.095926,-0.051664,0.119228,0.109646,0.109759,-0.181768,...,-0.119346,-0.139924,-0.272772,-0.386242,0.218397,0.275528,0.536367,0.256531,0.399271,-0.691504
15109,0.656685,0.437153,0.526300,0.198158,0.008960,-0.278656,-0.241767,-0.009052,-0.114615,-0.296820,...,-0.006828,-0.205763,0.011333,0.094089,0.197834,0.163011,0.098527,-0.196720,-0.258613,0.252062
15110,0.884435,0.957433,0.266550,-0.403613,-0.411181,0.208359,-0.093999,-0.114929,0.042168,-0.085458,...,0.028934,0.065975,0.331138,0.341994,0.015794,-0.213025,-0.156314,-0.068632,-0.226346,-0.478887


Normalizamos las características numéricas del vector caracteristico con scaler.

In [16]:
scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(features_to_normalize)
scaled_features

array([[ 0.02767268, -0.09061449,  0.12760617, ...,  0.38146715,
         0.32736809,  0.50969406],
       [-0.42056035, -0.31340308, -0.56157254, ..., -1.27720897,
        -0.92223662, -0.61069755],
       [ 1.20390308,  1.97734416,  0.11673799, ...,  0.39775803,
         1.7753855 ,  0.43969136],
       ...,
       [ 1.49797602,  1.25828645,  1.67653717, ..., -0.70986036,
        -0.64407615,  0.58798927],
       [ 1.94119238,  2.52523927,  0.92068435, ..., -0.31971855,
        -0.56523575, -0.59673322],
       [-0.5692139 , -0.4313095 , -1.05988482, ..., -0.04453381,
        -0.82135577,  0.39493294]])

Combinamos las características normalizadas con las otras columnas en una nueva dataframe con los datos normalizados

In [17]:
normalized_combined_data = pd.concat([combined_data[['track_id', 'track_name', 'track_artist', 'lyrics']], pd.DataFrame(scaled_features, columns=features_to_normalize.columns)], axis=1)
normalized_combined_data.head()


,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,0.027673,-0.090614,0.127606,-0.150135,0.623502,-0.535097,...,-0.462791,-0.418832,-0.738203,-0.548725,-0.092445,1.000128,0.013468,0.381467,0.327368,0.509694
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",-0.420560,-0.313403,-0.561573,0.624650,0.040056,0.406743,...,-0.405628,1.394239,0.373268,1.629134,0.436250,-2.116526,-1.002358,-1.277209,-0.922237,-0.610698
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,1.203903,1.977344,0.116738,-0.217151,-1.459183,-2.111601,...,-0.220164,-0.233633,-1.081571,0.151760,0.332779,-0.260502,0.715609,0.397758,1.775385,0.439691
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,-0.082804,0.522592,-0.526168,-1.009269,-0.327808,1.067858,...,-0.727788,-0.889488,-0.413088,-0.090049,-0.972259,-0.569831,-0.368295,0.574135,-1.580864,0.693064
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0.527152,-0.008320,0.129732,-0.465533,-0.339895,-0.792978,...,-0.566328,-1.307673,0.666147,0.262084,0.573582,-0.025321,-0.554712,-0.591808,-0.495331,0.528434


Limpieza de datos a esta nueva dataframe:

- **Verificar si hay valores NaN en el DataFrame**

- **Mostrar las filas con al menos un valor NaN**

- **Mostrar el número total de filas con al menos un valor NaN**

In [18]:
nan_check = normalized_combined_data.isna().any(axis=1)

rows_with_nan = normalized_combined_data[nan_check]
print(f"Filas con al menos un valor NaN:\n{rows_with_nan}")

total_rows_with_nan = rows_with_nan.shape[0]
print(f"Número total de filas con al menos un valor NaN: {total_rows_with_nan}")


Filas con al menos un valor NaN:
Empty DataFrame
Columns: [track_id, track_name, track_artist, lyrics, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26, f27, f28, f29, f30, f31, f32, f33, f34, f35, f36, f37, f38, f39, f40, f41, f42, f43, f44, f45, f46, f47, f48, f49, f50, f51, f52, f53, f54, f55, f56, f57, f58, f59, f60, f61, f62, f63, f64, f65, f66, f67, f68, f69, f70, f71, f72, f73, f74, f75, f76, f77, f78, f79, f80, f81, f82, f83, f84, f85, f86, f87, f88, f89, f90, f91, f92, f93, f94, f95, f96, ...]
Index: []

[0 rows x 364 columns]
Número total de filas con al menos un valor NaN: 0


Verficamos nuestra dataframe limpia.

In [19]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,0.027673,-0.090614,0.127606,-0.150135,0.623502,-0.535097,...,-0.462791,-0.418832,-0.738203,-0.548725,-0.092445,1.000128,0.013468,0.381467,0.327368,0.509694
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",-0.420560,-0.313403,-0.561573,0.624650,0.040056,0.406743,...,-0.405628,1.394239,0.373268,1.629134,0.436250,-2.116526,-1.002358,-1.277209,-0.922237,-0.610698
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,1.203903,1.977344,0.116738,-0.217151,-1.459183,-2.111601,...,-0.220164,-0.233633,-1.081571,0.151760,0.332779,-0.260502,0.715609,0.397758,1.775385,0.439691
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,-0.082804,0.522592,-0.526168,-1.009269,-0.327808,1.067858,...,-0.727788,-0.889488,-0.413088,-0.090049,-0.972259,-0.569831,-0.368295,0.574135,-1.580864,0.693064
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0.527152,-0.008320,0.129732,-0.465533,-0.339895,-0.792978,...,-0.566328,-1.307673,0.666147,0.262084,0.573582,-0.025321,-0.554712,-0.591808,-0.495331,0.528434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15107,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.953316,-0.481311,-1.291002,0.412203,1.097937,-0.027609,...,-0.659309,-0.754013,0.121373,0.065444,0.450124,0.691254,1.331063,0.604111,-0.211062,-1.509414
15108,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.658866,-0.376239,-0.318478,-0.427318,-0.283321,-0.164434,...,-0.410647,-0.462796,-0.945035,-1.403417,0.643448,0.805906,1.664743,0.670699,0.963397,-0.941343
15109,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",1.497976,1.258286,1.676537,0.753935,0.078347,-0.992855,...,-0.033475,-0.684713,0.014484,0.249745,0.570504,0.409526,0.184281,-0.709860,-0.644076,0.587989
15110,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",1.941192,2.525239,0.920684,-1.224058,-1.370378,0.784535,...,0.086404,0.231209,1.094575,1.102961,-0.075265,-0.915191,-0.677409,-0.319719,-0.565236,-0.596733


Esta parte parece ser innecesaria, pero se realiza y es que a partir del vector caracteristico se normalizara ahora con la columna lyrycs de la cancion para obetener un vector caracteristico mas exacto.

In [20]:
# Seleccionar las características numéricas normalizadas
numeric_features = normalized_combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
numeric_features

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0.027673,-0.090614,0.127606,-0.150135,0.623502,-0.535097,-0.474290,0.199920,-0.597988,0.287060,...,-0.462791,-0.418832,-0.738203,-0.548725,-0.092445,1.000128,0.013468,0.381467,0.327368,0.509694
1,-0.420560,-0.313403,-0.561573,0.624650,0.040056,0.406743,0.789381,1.139786,0.000501,-0.541132,...,-0.405628,1.394239,0.373268,1.629134,0.436250,-2.116526,-1.002358,-1.277209,-0.922237,-0.610698
2,1.203903,1.977344,0.116738,-0.217151,-1.459183,-2.111601,-1.192726,-0.628959,-1.088399,-0.482255,...,-0.220164,-0.233633,-1.081571,0.151760,0.332779,-0.260502,0.715609,0.397758,1.775385,0.439691
3,-0.082804,0.522592,-0.526168,-1.009269,-0.327808,1.067858,1.201576,0.460338,-1.320410,-0.043198,...,-0.727788,-0.889488,-0.413088,-0.090049,-0.972259,-0.569831,-0.368295,0.574135,-1.580864,0.693064
4,0.527152,-0.008320,0.129732,-0.465533,-0.339895,-0.792978,-0.209013,0.002260,-0.416470,0.251394,...,-0.566328,-1.307673,0.666147,0.262084,0.573582,-0.025321,-0.554712,-0.591808,-0.495331,0.528434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15107,-0.953316,-0.481311,-1.291002,0.412203,1.097937,-0.027609,0.446505,0.446332,0.325868,0.223313,...,-0.659309,-0.754013,0.121373,0.065444,0.450124,0.691254,1.331063,0.604111,-0.211062,-1.509414
15108,-0.658866,-0.376239,-0.318478,-0.427318,-0.283321,-0.164434,0.436081,0.379368,0.359862,-0.776616,...,-0.410647,-0.462796,-0.945035,-1.403417,0.643448,0.805906,1.664743,0.670699,0.963397,-0.941343
15109,1.497976,1.258286,1.676537,0.753935,0.078347,-0.992855,-0.896475,-0.067871,-0.490106,-1.209213,...,-0.033475,-0.684713,0.014484,0.249745,0.570504,0.409526,0.184281,-0.709860,-0.644076,0.587989
15110,1.941192,2.525239,0.920684,-1.224058,-1.370378,0.784535,-0.351015,-0.466801,0.103816,-0.414490,...,0.086404,0.231209,1.094575,1.102961,-0.075265,-0.915191,-0.677409,-0.319719,-0.565236,-0.596733


In [21]:
# Crear un nuevo vectorizador para las letras
text_vectorizer = TfidfVectorizer()
text_features = text_vectorizer.fit_transform(normalized_combined_data["lyrics"])
text_features

<15112x92553 sparse matrix of type '<class 'numpy.float64'>'
	with 1758442 stored elements in Compressed Sparse Row format>

In [22]:
scaler = preprocessing.StandardScaler()
scaled_numeric_features = scaler.fit_transform(numeric_features)
scaled_numeric_features

array([[ 0.02767268, -0.09061449,  0.12760617, ...,  0.38146715,
         0.32736809,  0.50969406],
       [-0.42056035, -0.31340308, -0.56157254, ..., -1.27720897,
        -0.92223662, -0.61069755],
       [ 1.20390308,  1.97734416,  0.11673799, ...,  0.39775803,
         1.7753855 ,  0.43969136],
       ...,
       [ 1.49797602,  1.25828645,  1.67653717, ..., -0.70986036,
        -0.64407615,  0.58798927],
       [ 1.94119238,  2.52523927,  0.92068435, ..., -0.31971855,
        -0.56523575, -0.59673322],
       [-0.5692139 , -0.4313095 , -1.05988482, ..., -0.04453381,
        -0.82135577,  0.39493294]])

In [23]:
# Combinar las características normalizadas con las otras columnas
combined_features = np.hstack((scaled_numeric_features, text_features.toarray()))
combined_features

array([[ 0.02767268, -0.09061449,  0.12760617, ...,  0.        ,
         0.        ,  0.        ],
       [-0.42056035, -0.31340308, -0.56157254, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.20390308,  1.97734416,  0.11673799, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.49797602,  1.25828645,  1.67653717, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.94119238,  2.52523927,  0.92068435, ...,  0.        ,
         0.        ,  0.        ],
       [-0.5692139 , -0.4313095 , -1.05988482, ...,  0.        ,
         0.        ,  0.        ]])

Obtenemos el mismo vector caracteristico no hay diferencia.

In [24]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,sugar honey ice & tea - Edit,Bring Me The Horizon,Count down the thunder I think we're too close...,0.027673,-0.090614,0.127606,-0.150135,0.623502,-0.535097,...,-0.462791,-0.418832,-0.738203,-0.548725,-0.092445,1.000128,0.013468,0.381467,0.327368,0.509694
1,0ZfM5XfJTtFPhOxAERRnNY,The Look,Roxette,"One, two, three, four Walkin' like a man Hitti...",-0.420560,-0.313403,-0.561573,0.624650,0.040056,0.406743,...,-0.405628,1.394239,0.373268,1.629134,0.436250,-2.116526,-1.002358,-1.277209,-0.922237,-0.610698
2,0zG6VemmYlMhAK5MNvBeP7,No Sleep,Bougenvilla,We don't get no sleep We don't get no sleep If...,1.203903,1.977344,0.116738,-0.217151,-1.459183,-2.111601,...,-0.220164,-0.233633,-1.081571,0.151760,0.332779,-0.260502,0.715609,0.397758,1.775385,0.439691
3,0ZGLuduCPjgWY1n85ykgMe,Back In the Saddle,Aerosmith,I'm back I'm back in the saddle again I'm back...,-0.082804,0.522592,-0.526168,-1.009269,-0.327808,1.067858,...,-0.727788,-0.889488,-0.413088,-0.090049,-0.972259,-0.569831,-0.368295,0.574135,-1.580864,0.693064
4,0ZgTez1pojM5aCxsV0hrC9,No Guarantee - Remix Version,Chico DeBarge,Got a one room apartment To a little tiny spac...,0.527152,-0.008320,0.129732,-0.465533,-0.339895,-0.792978,...,-0.566328,-1.307673,0.666147,0.262084,0.573582,-0.025321,-0.554712,-0.591808,-0.495331,0.528434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15107,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.953316,-0.481311,-1.291002,0.412203,1.097937,-0.027609,...,-0.659309,-0.754013,0.121373,0.065444,0.450124,0.691254,1.331063,0.604111,-0.211062,-1.509414
15108,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.658866,-0.376239,-0.318478,-0.427318,-0.283321,-0.164434,...,-0.410647,-0.462796,-0.945035,-1.403417,0.643448,0.805906,1.664743,0.670699,0.963397,-0.941343
15109,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",1.497976,1.258286,1.676537,0.753935,0.078347,-0.992855,...,-0.033475,-0.684713,0.014484,0.249745,0.570504,0.409526,0.184281,-0.709860,-0.644076,0.587989
15110,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",1.941192,2.525239,0.920684,-1.224058,-1.370378,0.784535,...,0.086404,0.231209,1.094575,1.102961,-0.075265,-0.915191,-0.677409,-0.319719,-0.565236,-0.596733


Esta parte es necesaria para la query de modo que crearemos un vector caracteristico para esta consulta, necesariamente la misma cantidad de caracteristicas que de las canciones.

In [27]:
def extract_and_normalize_features(audio_path, normalized_combined_data):
    y, sr = librosa.load(audio_path)

    # Extraer características de audio (aquí puedes elegir las características que deseas)
    features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=360)  # Ahora, extraer 12000 coeficientes MFCC

    # Asegurarse de que las características tengan el mismo tamaño que las del DataFrame normalized_combined_data
    max_length = normalized_combined_data.shape[1] - 4

    # Ajustar el tamaño de las características (rellenando con ceros si es necesario)
    if features.shape[1] < max_length:
        pad_width = max_length - features.shape[1]
        features = np.pad(features, pad_width=((0, 0), (0, pad_width)))

    # Crear un DataFrame con las características del audio
    audio_features = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(1, 361)])

    # Convertir el DataFrame de características de audio a un array NumPy
    audio_features_array = audio_features.values

    # Normalizar el vector de características de audio
    scaled_audio_features = scaler.transform(audio_features_array)

    # Convertir el array NumPy a un DataFrame
    scaled_audio_features = pd.DataFrame(scaled_audio_features, columns=[f'feature_{i}' for i in range(1, 361)])

    return scaled_audio_features


Consulta 2

In [28]:
audio_path="../query2.wav"
scaled_audio_features2=extract_and_normalize_features(audio_path, normalized_combined_data)
scaled_audio_features2

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_351,feature_352,feature_353,feature_354,feature_355,feature_356,feature_357,feature_358,feature_359,feature_360
0,-435.298279,-432.778809,-491.277191,-527.434998,-526.483032,-522.620422,-521.851257,-522.604980,-518.619934,-514.677246,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
1,61.816315,63.625111,27.193474,4.778519,4.786145,8.438463,10.664425,8.868388,7.793067,8.470732,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
2,-55.187035,-55.401085,-27.905628,-4.767743,0.650598,-1.385258,-1.125632,-1.032184,-3.864204,-4.688291,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
3,-52.830360,-52.119423,-28.434399,-6.015786,-4.101000,-3.008466,-1.221179,-0.475851,0.582900,0.213655,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
4,10.062284,12.080715,7.896958,1.653944,-2.521240,0.098991,2.873945,0.940562,-0.879641,-1.985665,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-1.098422,-0.702476,-0.584366,-0.297125,0.215557,0.682936,1.958951,1.384546,0.835650,2.318981,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
124,-1.959123,-1.521884,0.315836,0.297115,1.229116,0.066248,0.536730,0.264844,0.020233,-1.256701,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
125,0.172777,1.210672,0.571567,0.673255,0.338192,-0.625111,-0.913269,-0.129975,0.376198,-1.778493,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
126,0.954302,1.602627,-0.973606,-0.442813,0.959123,-0.101996,0.427108,0.916409,2.005571,0.559360,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17


Consulta 1

In [29]:
audio_path="../query.wav"
scaled_audio_features=extract_and_normalize_features(audio_path, normalized_combined_data)

scaled_audio_features

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_351,feature_352,feature_353,feature_354,feature_355,feature_356,feature_357,feature_358,feature_359,feature_360
0,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
1,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
2,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
3,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
4,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
124,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
125,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
126,2.821107e-18,-7.993136e-18,-9.403689e-19,-4.701845e-19,-1.034406e-17,1.387044e-17,6.582582e-18,1.504590e-17,1.175461e-17,-1.880738e-17,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17


Consulta 3

In [30]:
audio_path="../query3.wav"
scaled_audio_features3=extract_and_normalize_features(audio_path, normalized_combined_data)

scaled_audio_features3

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_351,feature_352,feature_353,feature_354,feature_355,feature_356,feature_357,feature_358,feature_359,feature_360
0,-452.724274,-410.642242,-382.672241,-369.403137,-319.774872,-283.140106,-261.333923,-245.908600,-241.301178,-234.242737,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
1,16.924856,37.878498,53.071121,54.236336,49.737312,47.321243,46.468109,50.987701,49.823235,45.855473,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
2,2.595886,-12.398136,-20.443975,-33.565346,-72.241028,-86.934692,-90.179291,-91.479698,-90.874260,-83.739441,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
3,0.188916,7.635675,9.030219,5.604999,18.592773,29.668083,39.156750,45.619164,54.013000,61.148346,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
4,9.110559,25.848841,28.894205,29.356882,38.982628,33.725739,24.050903,20.416084,21.267445,16.894958,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1.014865,0.491443,0.105006,-1.209732,-2.149136,-1.459698,-1.026202,-0.505115,-2.157951,-2.793029,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
124,-1.337445,0.952938,1.291588,0.369337,-0.984945,0.211321,0.724087,-0.481273,-2.745622,-2.903449,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
125,-0.919728,-2.540533,-2.342331,-0.719841,1.260162,2.068654,0.259312,-0.320774,-1.088140,-1.899452,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17
126,0.900744,-3.300434,-3.379250,-2.151307,-0.587406,1.479919,0.253590,1.374237,1.542091,0.965557,...,9.873873e-18,-2.256885e-17,1.034406e-17,3.761476e-18,1.316516e-17,1.692664e-17,-1.551609e-17,7.522951e-18,-1.128443e-17,-1.222480e-17


EL modelo KNN search se implementa con ayudade sklearn que nos proporciona la facilidad con los modelos de dos arrays se logra obtener los k vecinos mas cercanos

# Sin cola de prioridad

In [31]:
from sklearn.neighbors import NearestNeighbors

def knn_search(query_audio_features, reference_features, k=5, metric='cosine'):
    # Inicializar el modelo KNN
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extraer las características numéricas para el modelo KNN, 4 primeras columnas del data frame
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convertir query_audio_features a un array NumPy
    query_features_numeric = query_audio_features.to_numpy()

    # Utilizar el modelo para encontrar los k vecinos más cercanos para la query del audio
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Obtener las k canciones más cercanas utilizando los índices
    nearest_songs = reference_features.iloc[indices[0], :4]  # Obtener solo las primeras 4 columnas

    return nearest_songs


# Con cola de prioridad

In [32]:
from queue import PriorityQueue
from sklearn.neighbors import NearestNeighbors

def knn_search_priority_queue(query_audio_features, reference_features, k=5, metric='cosine'):
    # Initialize the KNN model
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extract the numeric features for the KNN model, excluding the first 4 columns
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convert query_audio_features to a NumPy array
    query_features_numeric = query_audio_features.to_numpy()

    # Use the model to find the k nearest neighbors for the audio query
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Create a priority queue to store the k nearest neighbors
    pq = PriorityQueue()

    # Iterate over the indices and distances to add the neighbors to the priority queue
    for i in range(len(indices[0])):
        index = indices[0][i]
        distance = distances[0][i]
        pq.put((distance, index))

    # Get the k nearest neighbors from the priority queue
    nearest_songs = []
    while not pq.empty():
        distance, index = pq.get()
        nearest_songs.append(reference_features.iloc[index, :4])  # Get only the first 4 columns

    return nearest_songs


## Query 1


In [33]:
k_nearest_songs = knn_search(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
5532,3G69vJMWsX6ZohTykad2AU,One,U2,Is it getting better Or do you feel the same? ...
6936,3zbSYBDUbDEbqUIlCmOw09,Procura,ChiChi Peralta,Procura seducirme muy despacio Y no reparo de ...
1482,1n6pgjMma0nsl4jyMzmjxQ,Watch Out Now,The Beatnuts,"""Watch out now!"" Yea, yea.. uh-uh! Mm-mmm! Get..."
3257,2g8KDyKv8lA0bhmK8je3Zc,That's My Attitude,Troop,Intro: That's my attitude Hello Who is this Ba...
2048,1UO1yyMxEAYmYYQgIQJKMm,"Alright - 7"" House Mix With Rap",Janet Jackson,Alright Alright Alright Alright Alright Alrigh...


In [34]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
5532,3G69vJMWsX6ZohTykad2AU,One,U2,Is it getting better Or do you feel the same? ...
6936,3zbSYBDUbDEbqUIlCmOw09,Procura,ChiChi Peralta,Procura seducirme muy despacio Y no reparo de ...
1482,1n6pgjMma0nsl4jyMzmjxQ,Watch Out Now,The Beatnuts,"""Watch out now!"" Yea, yea.. uh-uh! Mm-mmm! Get..."
3257,2g8KDyKv8lA0bhmK8je3Zc,That's My Attitude,Troop,Intro: That's my attitude Hello Who is this Ba...
2048,1UO1yyMxEAYmYYQgIQJKMm,"Alright - 7"" House Mix With Rap",Janet Jackson,Alright Alright Alright Alright Alright Alrigh...


## Query 2

In [35]:
k_nearest_songs = knn_search(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
8873,4TnUKixNWMfajncgdSwFoi,Amarillo By Morning,George Strait,"Amarillo by morning, up from San Antone Everyt..."
11797,64Ret7Tf2M8pDE4aqbW2tX,One,Metallica,NA I can't remember anything Can't tell if thi...
978,1h22h211rde6dDCeW2hXpN,"Up Where We Belong - From ""An Officer And A Ge...",Joe Cocker,Who knows what tomorrow brings In a world few ...
614,1chu8wc9zbNqTQ2ahROys1,My Sweet Lord,George Harrison,"My sweet Lord Mmm, my Lord Mmm, my Lord I real..."
13742,6WunLSy2T9dGLdW30ibt8s,Por Tus Pujidos Nos Hallaron,Akwid,La conocí desde morro cuando era inocente Noma...


In [36]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
8873,4TnUKixNWMfajncgdSwFoi,Amarillo By Morning,George Strait,"Amarillo by morning, up from San Antone Everyt..."
11797,64Ret7Tf2M8pDE4aqbW2tX,One,Metallica,NA I can't remember anything Can't tell if thi...
978,1h22h211rde6dDCeW2hXpN,"Up Where We Belong - From ""An Officer And A Ge...",Joe Cocker,Who knows what tomorrow brings In a world few ...
614,1chu8wc9zbNqTQ2ahROys1,My Sweet Lord,George Harrison,"My sweet Lord Mmm, my Lord Mmm, my Lord I real..."
13742,6WunLSy2T9dGLdW30ibt8s,Por Tus Pujidos Nos Hallaron,Akwid,La conocí desde morro cuando era inocente Noma...


## Query 3

In [37]:
k_nearest_songs = knn_search(scaled_audio_features3, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
12173,6CNscPXOoE9fln0ShewnaB,Stop! - Player & Remady Swiss Mix,DJ Antoine,Säg mer jetzt einisch meh Säg mer jetzt wievie...
15030,7IzzrvxEKHgdxn3DIAlrfe,Nivel,Lucho SSJ,"NA Mi ganga anda whippin', bajan dos o mil qui..."
922,1GfettdrSIqEky8VvxpxAo,Blue Ocean Floor,Justin Timberlake,Frequencies so low Heart on a string A string ...
10171,5gJcMDLHDllFMhSPByZhhE,I Believe I Can Fly - Radio Edit,R. Kelly,I used to think that I could not go on And lif...
5379,3e5eR2DBoHD2qR4HN3YmTv,Whole Wide World,Bahamas,"When I was a young boy, my mama said to me The..."


In [38]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features3, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
12173,6CNscPXOoE9fln0ShewnaB,Stop! - Player & Remady Swiss Mix,DJ Antoine,Säg mer jetzt einisch meh Säg mer jetzt wievie...
15030,7IzzrvxEKHgdxn3DIAlrfe,Nivel,Lucho SSJ,"NA Mi ganga anda whippin', bajan dos o mil qui..."
922,1GfettdrSIqEky8VvxpxAo,Blue Ocean Floor,Justin Timberlake,Frequencies so low Heart on a string A string ...
10171,5gJcMDLHDllFMhSPByZhhE,I Believe I Can Fly - Radio Edit,R. Kelly,I used to think that I could not go on And lif...
5379,3e5eR2DBoHD2qR4HN3YmTv,Whole Wide World,Bahamas,"When I was a young boy, my mama said to me The..."


## Range Search
El range search utilizamos el modelo KNN y obtenemos resultados interesantes.

In [39]:
from sklearn.neighbors import RadiusNeighborsClassifier


def range_search(query_features=pd.DataFrame(), reference_features=pd.DataFrame(), radius=int()):
    # Solo trabajaremos con el vector característico de las canciones
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)

    # Convertimos el dataframe de la consulta a un array numpy
    query_features_numeric = query_features.to_numpy()

    # Creamos el modelo RadiusNeighborsClassifier
    knn_model = RadiusNeighborsClassifier(radius=radius, metric='euclidean')

    # Entrenamos al modelo con el vector característico de las canciones
    knn_model.fit(reference_features_numeric, reference_features['track_id'])

    # Buscamos vecinos dentro del radio para cada punto de la consulta
    neighbor_indices = knn_model.radius_neighbors(query_features_numeric, return_distance=False)

    # Extraemos información de las canciones encontradas
    song_data = []
    unique_tracks = set()

    for query_index, indices in enumerate(neighbor_indices):
        for neighbor_index in indices:
            track_id = reference_features['track_id'].iloc[neighbor_index]
            if track_id not in unique_tracks:
                unique_tracks.add(track_id)
                song_data.append({
                    'track_id': track_id,
                    'track_name': reference_features['track_name'].iloc[neighbor_index],
                    'track_artist': reference_features['track_artist'].iloc[neighbor_index],
                })

    return song_data

## Query 1
Query 1 normalizada mala diccion y bajo volumen. 

In [42]:
radius = 10
indices_within_radius = range_search(scaled_audio_features, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,3x4q0AuhGdiPi1jvCIFACQ,This Place,Logan Pollard
1,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib
2,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music
3,6CCcI6EsbZ15NsCAMJAkpC,Hit the Spot,Miles Casper
4,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill
5,5i3qlYyzV6nc4EvCrS1YVY,Manifest,Zayvin Lakin


## Query 2
Query 2 diccion aceptable y volumen aceptable

In [48]:
radius = 15
indices_within_radius = range_search(scaled_audio_features2, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,3x4q0AuhGdiPi1jvCIFACQ,This Place,Logan Pollard
1,1arQh74CO1oQLKrO7el3FV,Devil's Pie,D'Angelo
2,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr
3,1h20yRsf5WeuHbohrlfcCt,Me Or The Papes,Jeru The Damaja
4,1K6oIHSzqcLpToFiwIDSI7,Memories,Cypress Hill
...,...,...,...
284,7AwCvK8ONNNNFe4Fsqv2fD,Bata Bata 2,Latin Fresh
285,7HkustDHNYWwUdNO9ootCs,Semillas,La Banda Baston
286,7IfywdSxY0okFji12beS6Y,Now I Feel Ya,Scarface
287,2am14wFmbow4gZXQy1Pwyh,Can't Stop - Remix,After 7


## Query 3
La query 3 es mas clara mejor diccion y pronunciacion mejor resultado que la query 1, demostrando que si la consulta es una que no tiene una buena diccion y volumen bajo, obtenemos malos resultados.

In [50]:
radius = 15
indices_within_radius = range_search(scaled_audio_features3, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,3x4q0AuhGdiPi1jvCIFACQ,This Place,Logan Pollard
1,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill
2,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib
3,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music
4,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr
5,1K6oIHSzqcLpToFiwIDSI7,Memories,Cypress Hill
6,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr
7,5i3qlYyzV6nc4EvCrS1YVY,Manifest,Zayvin Lakin
8,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor
9,6CCcI6EsbZ15NsCAMJAkpC,Hit the Spot,Miles Casper


## Aclaraciones

La data frame tiene una cancion pero de distintas versiones y tiempo, cuenta con detalles que otros no tienen.

In [55]:
# Filtrar el DataFrame original para obtener todas las instancias de la canción con track_id específico
all_instances = songs[songs['track_id'] == '05xbEV5M8DAQmhFKKlHaDd']

# Contar el número de instancias
num_instances = all_instances.shape[0]

# Imprimir el resultado
print(f"La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite {num_instances} veces.")


La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite 1 veces.


In [63]:
# Identificar posibles versiones duplicadas basadas en ciertas columnas (por ejemplo, artist_name y title)
possible_duplicates = songs[songs.duplicated(['track_name', 'track_album_name'], keep=False)]

# Mostrar el DataFrame con posibles versiones duplicadas
possible_duplicates


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
10,00hdjyXt6MohKnCyDmhxOL,Una Vaina Loca,Fuego,Fuego Uoh uoh uoh La musiica del futuroo Yeah!...,1,01nV3KuocS1NJHTsJbPkTO,Una Vaina Loca,2011-11-02,MIX LATIN POP°,6IS6XTdbS9qJZgfjNKgpB8,...,-5.589,1,0.0361,0.11400,0.000080,0.0620,0.642,117.009,188213,es
16,00LfFm08VWeZwB0Zlm24AT,Suga Suga,Baby Bash,"So tight, so fly You got me lifted, you got me...",8,2bAkYizbM1rMbOUHpfAV9z,Tha Smokin' Nephew,2003-01-01,90s/00s Hip Hop & RnB,0Ar0Ng9DlAWZtSPBvOQgOa,...,-3.041,0,0.2680,0.68800,0.000008,0.0841,0.535,82.331,239027,en
41,01auWCuHXubb8KPZ0IXF4S,Get Get Down - Radio Edit,Merk & Kremont,"Down, down do-down, now ge-get",37,1P5jv5F4eW3TPh5k10I3Uc,Get Get Down,2015-06-05,Alex Workout,7fRlDx99wBg6i45OCvxwKY,...,-4.087,0,0.0764,0.00724,0.013400,0.4680,0.332,128.032,181875,pl
42,01Bw3xiZ5PQfn0btMWICaQ,Beautiful Life,Lost Frequencies,"Anywhere you are and you'll go, I follow Mount...",30,2WGceQjphDCN6OdXxNBH6x,Beautiful Life,2016-06-03,Tropical Beats,37i9dQZF1DX1bDvyV0IkZ3,...,-6.684,0,0.0376,0.12700,0.034700,0.1390,0.339,119.979,161464,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18384,7yCPwWs66K8Ba5lFuU2bcx,All The Small Things,blink-182,"All the small things True care, truth brings I...",21,5qt11cWjSs5Gbqj2Wyfu38,Enema Of The State,1999-01-01,keg party jukebox,4PjiT7SWnUsTTTvKOuR76f,...,-4.764,1,0.0570,0.01220,0.000000,0.5470,0.695,148.599,168000,tl
18395,7ykaUgkdQWJLsMuOymTV2A,If Not For You,George Harrison,"If not for you Babe, I couldn't even find the ...",59,4I4xtHaIFOzhZfp1NIHkY6,All Things Must Pass (Remastered),1970-11-27,I didn’t know perm stood for permanent (wave),3e6gYPyrTbaB8BWgSHCt5j,...,-6.886,1,0.0317,0.21900,0.003990,0.0820,0.867,116.119,210720,en
18402,7yqU5HODQU91zj8ahUaUX1,I Wanna Dance with Somebody (Who Loves Me),Whitney Houston,Clock strikes upon the hour And the sun begins...,2,1wvEC2yY7koRQYebhrj1ZY,Whitney,1987-06-02,80s Pop & Rock Hits and Album Tracks,0XOIK4m26aeYSD61E5nSVW,...,-8.824,1,0.0453,0.20700,0.000307,0.0888,0.867,118.814,291293,en
18418,7zdoYASH8iYGR1CQV3m8dY,SOS,McClenney,"Hey, hey, oh, oh Hey, hey, oh, oh Hey, hey, oh...",5,7CHix0EuA9xs2RY2sFi5Pv,SOS,2019-02-19,I Love My Neo-Soul,37i9dQZF1DX44dZ4p5QLf4,...,-13.628,0,0.0390,0.57200,0.003940,0.1220,0.555,72.950,187242,en


In [70]:
# Seleccionar la canción específica (cambiar 'track_name' y 'track_album_name' según tus necesidades)
cancion_seleccionada = songs[(songs['track_name'] == 'Poison')]

# Contar duplicados para la canción seleccionada
duplicados_cancion_seleccionada = cancion_seleccionada.shape[0]

# Imprimir el resultado
print(f"La canción seleccionada se repite {duplicados_cancion_seleccionada} veces.")

cancion_seleccionada


La canción seleccionada se repite 13 veces.


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
237,06yHcjr58IKJnfEK7ko3sD,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",36,3ExqNzkCrQkkknQbrJFqMd,RnB 1990's,2019-07-19,CSR 103:9 (GTA: SA),4sr2xEhXQR5VuZ0LZX8TQ8,...,-10.530,0,0.2100,0.00195,0.004170,0.6330,0.775,111.815,261853,en
622,0dDifktDnbzmGwliFzI0Ld,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,5AR2bbUUDI2XEMarSbs8Cn,Bell Biv DeVoe Greatest Hits (Remastered),2000,New Jack Swing,3ykXidKLz1eYPvuGoFlD1e,...,-9.376,0,0.2100,0.00206,0.001830,0.6340,0.783,111.879,261627,en
1705,0RnDu3eYJqbFKz6MHv2ajd,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",48,2QC1IsQIUNdEz0zgWanPkN,20th Century Masters: The Millennium Collectio...,2002-01-01,New Jack Swing - 90s R&B fused w Hip Hop,79xd4wnVuKZK4rJMsL2wPa,...,-6.365,0,0.1920,0.00215,0.002680,0.6270,0.802,111.857,262027,en
3433,1iQpStZpzBb5TOuTDKJKdj,Poison,Alice Cooper,"Your cruel device Your blood, like ice One loo...",59,0ztezUxR9YMkIP1SFRSnV1,Alice Cooper Classicks,1989,80s / Classic Rock,29dTrOurPDrMcrnio2q6hZ,...,-7.786,0,0.1090,0.05150,0.000333,0.1250,0.268,118.631,269600,en
4878,231mi5BkHyJHxQre25S6oS,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,0PPAtp7bE7Itwl0UtG1mXc,20 #1's: 90's R&B,2015-06-15,90's NEW JACK SWING,0dmInkymNnOTWvEFamSNzb,...,-7.312,0,0.2030,0.00188,0.001570,0.5990,0.761,111.871,262053,en
5419,2donM8sTsqYtRpSAwRdPJQ,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",1,7r7Vb7w9Fc3kUDNgIlPIxk,Gold,2005-01-01,90s R&B - The BET Planet Groove/Midnight Love Mix,7LjRWGUdOmBdPG1W1bga4i,...,-7.526,0,0.2280,0.00615,0.005270,0.5030,0.676,111.912,262467,en
7429,39J10NL0mFTAdJbapoo2rC,Poison,Alice Cooper,"Your cruel device Your blood, like ice One loo...",57,7fP5RXmvSPfAY1EgffEPVx,The Best Of Alice Cooper,2007-05-05,Workday: Rock Classics,37i9dQZF1DX1lwxXvReaux,...,-6.096,0,0.1170,0.05240,0.000036,0.1110,0.305,118.794,270787,en
11776,4ZkVOb0mNnm3MFY2LeyqHY,Poison,Martin Garrix,NaN,2,2m4U0aYL6QDgbkL5GFbCmn,Poison,2015-11-02,Jeff Seid Electro House,2Tisn6NZbeofpc57Oc6MGo,...,-6.118,0,0.0363,0.00429,0.769000,0.3770,0.062,128.125,248447,NaN
13186,5NijSs5dAwaIybq1GaRTIe,Poison,Brent Faiyaz,I feel like there's no such thing as a relatio...,68,17dsllSewkPuKbkGWU4chD,Poison,2016-08-03,Urban Contemporary,4Pbs84EQbuAblxlp6Chz0d,...,-10.102,0,0.0650,0.71100,0.000905,0.0859,0.382,117.489,212992,en
